In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Bidirectional, Input

data = {
    "Hours": [1.2, 2.0, 3.5, 4.0, 5.3, 6.1, 7.2, 8.0, 9.5],
    "Scores": [18, 28, 33, 42, 57, 63, 77, 85, 97]
}

df = pd.DataFrame(data)
print("Sample Data:\n", df.head())

X = df[['Hours']].values
y = df['Scores'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=24
)

models = {
    "Linear Regression": LinearRegression(),
    "SVM": SVR(kernel='rbf', C=1.2, epsilon=0.2),
    "Random Forest": RandomForestRegressor(n_estimators=120, max_depth=5, random_state=24),
    "XGBoost": XGBRegressor(objective='reg:squarederror', n_estimators=120, learning_rate=0.12, random_state=24),
    "KNN": KNeighborsRegressor(n_neighbors=4),
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    results[name] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
nb = GaussianNB()
y_train_int = np.round(y_train)
nb.fit(X_train_scaled, y_train_int)
preds = nb.predict(X_test_scaled)
results["Naive Bayes (approx)"] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

mlp = MLPRegressor(hidden_layer_sizes=(70,30), max_iter=1800, random_state=24)
mlp.fit(X_train, y_train)
preds = mlp.predict(X_test)
results["MLP (Sklearn)"] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

X_train_rnn = np.expand_dims(X_train, axis=1)
X_test_rnn = np.expand_dims(X_test, axis=1)

def build_and_train(model, name):
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_rnn, y_train, epochs=180, verbose=0)
    preds = model.predict(X_test_rnn).flatten()
    results[name] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

rnn = Sequential([Input(shape=(1,1)), SimpleRNN(28), Dense(1)])
build_and_train(rnn, "RNN")

lstm = Sequential([Input(shape=(1,1)), LSTM(28), Dense(1)])
build_and_train(lstm, "LSTM")

bilstm = Sequential([Input(shape=(1,1)), Bidirectional(LSTM(28)), Dense(1)])
build_and_train(bilstm, "BiLSTM")

gru = Sequential([Input(shape=(1,1)), GRU(28), Dense(1)])
build_and_train(gru, "GRU")

bigru = Sequential([Input(shape=(1,1)), Bidirectional(GRU(28)), Dense(1)])
build_and_train(bigru, "BiGRU")

print("\nModel Performance (MSE & R2):")
print(pd.DataFrame(results).T)




Sample Data:
    Hours  Scores
0    1.2      18
1    2.0      28
2    3.5      33
3    4.0      42
4    5.3      57
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step

Model Performance (MSE & R2):
                              MSE         R2
Linear Regression       23.705790   0.649092
SVM                   2421.747575 -34.848237
Random Forest          805.130625 -10.918052
XGBoost                612.622009  -8.068418
KNN                   1480.062500 -20.908820
Naive Bayes (approx)   612.000000  -8.059211
MLP (Sklearn)           13.551711   0.799399
RNN                   6369.322266 -93.282730
LSTM                  6756.107910 -99.008179
BiLSTM                6071.234375 -88.870247
GRU                   6057.744629 -88.670563
BiGRU                 5667.806641 -82.898453
